In [45]:
import netCDF4 as nc
import numpy as np
import lidarSim as lidar
from kazrRet import *
import matplotlib.pyplot as plt

#'temp':(['nc','nz1'],tempCL),'qv':(['nc','nz'],qvCL),'rho':(['nc','nz'],rhoCL),'press':(['nc','nz'],pressCL)}

# read variables temp, qv, rho, press from the netcdf file
with nc.Dataset('profiles.nc','r') as f:
    qv=f.variables['qv'][:]
    press=f.variables['press'][:]
    temp=f.variables['temp'][:]
    rho=f.variables['rho'][:]

with nc.Dataset('active_obs_2020-08-11_05:10:00_subset.nc','r') as f:
    zKu_m=f.variables['zKu'][:]
    zKa_m=f.variables['zKa'][:]
    press_m=f.variables['press'][:]
    temp_m=f.variables['temp'][:]                
    dn_m=f.variables['dn'][:]
    qv_m=f.variables['qv'][:]
    height_m=f.variables['height'][:]
    print(f)
    
#plt.plot(qv[0,:],press[0,:])
#plt.plot(qv[:,:].mean(axis=0),press[:,:].mean(axis=0))
#plt.plot(qv_m[:,:].mean(axis=0),press_m[:,:].mean(axis=0),'*')
#plt.ylim(1e5,1e3)
#plt.figure()
#plt.plot(0.5*(temp[0,:-1]+temp[0,1:]),press[0,:])
#plt.plot(0.5*(temp[:,:-1]+temp[:,1:]).mean(axis=0),press[:,:].mean(axis=0),'*')
#plt.plot(temp_m[:,:].mean(axis=0),press_m[:,:].mean(axis=0))
#plt.ylim(1e5,1e3)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): nt(12886), nz(70), nz1(71)
    variables(dimensions): float64 iwc(nt, nz), float32 pnorm(nt, nz), float32 pmol(nt, nz), float64 height(nt, nz), float64 zKu(nt, nz), float64 zKa(nt, nz), float64 dn(nt, nz), float64 qv(nt, nz), float64 press(nt, nz), float64 temp(nt, nz), float64 tempf(nt, nz1)
    groups: 


In [48]:
lidSim=lidar
def get_index(dn01,k,zKu1,zST,lidSim):
    if zKu1[k]-10*dn01<=zST[0,0]:
        dn01=(zKu1[k]-zST[0,0])/10
    if zKu1[k]-10*dn01>=zST[0,-1]:
        dn01=(zKu1[k]-zST[0,-1])/10
    ifind = lidSim.bisection2(zST[0,:],zKu1[k]-10*dn01)
    if ifind>=400:
        ifind=399
    return ifind

print(qv.shape)
nt,nz=qv.shape
print(temp.shape)
ntm,nzm=qv_m.shape
tbL=[]
freqs=[13.6,35.5,94.0,183.31]
fisot=2.7
theta=53
umu=np.cos(np.deg2rad(theta))
#print(temp_m.shape)
#print(qv_m.shape)
zKuL=[]
iwcL=[]
import tqdm
for i in tqdm.tqdm(range(ntm)):
    kexttot_atm=np.zeros((nzm,4),float)
    kexttot_atm=np.zeros((nzm,4),float)
    emis=0.8+np.random.random()*0.2
    ebar=0.8+np.random.random()*0.2
    rho1d=press_m[i]/(287.05*temp_m[i])
    z0=height_m[i][0]
    hm=z0+np.arange(nz+1)*.240
    hmi=z0+np.arange(nz)*.240+0.120
    a=np.nonzero(zKu_m[i]>-25)
    iwc1=np.zeros(nzm,float)
    kextI=np.zeros((nzm,4),float)
    kscaI=np.zeros((nzm,4),float)
    asymI=np.zeros((nzm,4),float)
    for k in a[0]:
        dn01=0.5#dn_m[i,k]
        ifind=get_index(dn01,k,zKu_m[i,:],zST,lidSim) 
        iwc_ret=iwcST[0,ifind]*10**dn01
        iwc1[k]=iwc_ret
        kextI[k,:]=kextST[-4:,ifind]*10**dn01
        kscaI[k,:]=kscaST[-4:,ifind]*10**dn01
        asymI[k,:]=gST[-4:,ifind]
    ireturn=1
    for ik,freq in enumerate(freqs):
        for k in range(nzm):
            absair,abswv = lidar.gasabsr98(float(freq),temp_m[i][k],rho1d*qv_m[i][k]*1e-3,press_m[i][k],ireturn)
            kexttot_atm[k,ik]=(absair+abswv)
    kext=kexttot_atm[:,:]+kextI[:,:]
    salb=kexttot_atm.copy()*0+kscaI
    salb/=kext
    asym_fact=asymI
    tb1=[]
    tempi=np.interp(hm,height_m[i],temp_m[i])
    zKuL.append(np.interp(hmi,height_m[i],zKu_m[i]))
    iwcL.append(np.interp(hmi,height_m[i],iwc1))

    #print(heightmm.shape)
    #print(kexttot_atm.shape)
    #print(qv_m.shape,temp_m.shape)
    #print(height_m.shape)
    for ik in range(4):
        kexti=np.interp(hmi,height_m[i],kext[:,ik])
        salbi=np.interp(hmi,height_m[i],salb[:,ik])
        asym_facti=np.interp(hmi,height_m[i],asym_fact[:,ik])
        tbout = lidar.radtran(umu,tempi[0],tempi,hm,kexti,salbi,asym_facti,fisot,emis,ebar)
        tb1.append(tbout)
        if tbout!=tbout or tbout>300:
            stop
    tbL.append(tb1)
print(np.array(tbL).mean(axis=0))
import xarray as xr
ds=xr.Dataset({'tb':(['nt','n4'],np.array(tbL)),\
               'zKu':(['nt','nz'],np.array(zKuL)),
               'iwc':(['nt','nz'],np.array(iwcL))})
ds.to_netcdf('obs_wrf_randEmiss_inclined_2020-08-11_05:10:00_subset.nc',\
             encoding={'tb':{'zlib':True},'zKu':{'zlib':True},'iwc':{'zlib':True}})


(30, 64)
(30, 65)


100%|██████████| 12886/12886 [02:34<00:00, 83.27it/s]


[261.56617065 257.17493283 246.33440478 217.18212737]


In [47]:
print(qv.shape)
print(np.array(tbL).mean(axis=0))
print(temp.shape)
nt,nz=qv.shape
height=np.arange(nz+1)*0.240
tb2L=[]
freqs=[13.6,35.5,94.0,183.31]
fisot=2.7
umu=np.cos(np.deg2rad(theta))
kext_atm_class=[]
for i in range(nt):
    kexttot_atm=np.zeros((nz,4),float)
    kexttot_atm=np.zeros((nz,4),float)
    emis=0.9+np.random.random()*0.01
    ebar=0.9+np.random.random()*0.01
    rho1d=rho[i]
    ireturn=1
    for ik,freq in enumerate(freqs):
        for k in range(nz):
            absair,abswv = lidar.gasabsr98(float(freq),temp[i][k],rho1d*qv[i][k]*1e-3,press[i][k],ireturn)
            kexttot_atm[k,ik]=(absair+abswv)
    kext=kexttot_atm[:,:]
    kext_atm_class.append(kext)
    salb=kexttot_atm.copy()*0
    asym_fact=kexttot_atm.copy()*0
    tb1=[]
    for ik in range(4):
        tbout = lidar.radtran(umu,temp[i][0],temp[i],height,kext[:,ik],salb[:,ik],asym_fact[:,ik],fisot,emis,ebar)
        tb1.append(tbout)
        if tbout!=tbout:
            stop
    tb2L.append(tb1)
print(np.array(tbL).mean(axis=0))
print(np.array(tb2L).mean(axis=0))


(30, 64)
[262.81899027 261.02676512 253.91666394 217.31505984]
(30, 65)
[262.81899027 261.02676512 253.91666394 217.31505984]
[267.96338094 272.75645854 280.76924438 217.87990265]
